## Script DDL – Estrutura do Banco

Estrutura contendo os comandos de criação da estrutura do banco de dados (tabelas, tipos de dados, restrições etc.).

## Criação das tabelas no Banco OracleDB

Este comando SQL cria a tabela ota_bus_contact_data no banco de dados Oracle. A tabela armazena informações detalhadas sobre os clientes, funcionando como uma tabela de dimensão em um modelo de dados de **Data Warehouse**.

In [0]:
%sql
CREATE TABLE ota_bus_contact_data (
    fk_contact                    VARCHAR2(20) PRIMARY KEY,
    customer_code                 INTEGER,
    customer_name                 VARCHAR2(255),
    birth_date                    DATE,
    gender                        VARCHAR2(20),
    marital_status                VARCHAR2(20),
    email                         VARCHAR2(100),
    phone_number                  VARCHAR2(20),
    address                       VARCHAR2(255),
    neighborhood                  VARCHAR2(100),
    city                          VARCHAR2(100),
    state                         VARCHAR2(50),
    postal_code                   VARCHAR2(20),
    country                       VARCHAR2(50),
    customer_group                VARCHAR2(50),
    income                        NUMBER(15, 2),
    age_group                     VARCHAR2(20),
    occupation                    VARCHAR2(50),
    customer_status               VARCHAR2(20),
    last_update_date              DATE,
    active_record_indicator       CHAR(1)
);

In [0]:
import pandas as pd

df_sand_bus_company = spark.table("bus.bronze.ota_bus_ticket_sales")

unique_departure = (
    df_sand_bus_company
    .select("fk_departure_ota_bus_company")
    .dropna(subset=["fk_departure_ota_bus_company"])
    .distinct()
    .toPandas()["fk_departure_ota_bus_company"]
)

unique_return = (
    df_sand_bus_company
    .select("fk_return_ota_bus_company")
    .dropna(subset=["fk_return_ota_bus_company"])
    .distinct()
    .toPandas()["fk_return_ota_bus_company"]
)

unique_companies = pd.Series(
    list(unique_departure) + list(unique_return)
).drop_duplicates().reset_index(drop=True)

companies_df = pd.DataFrame({
    "bus_company_id": range(1, len(unique_companies) + 1),
    "bus_company_name": unique_companies,
    "bus_company_cnpj": [
        f"{i:08d}{i:04d}"[:14]
        for i in range(1, len(unique_companies) + 1)
    ]
})

df_gold_bus_company = spark.createDataFrame(companies_df)

df_gold_bus_company.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bus.gold.ota_bus_company")

In [0]:
%sql
CREATE TABLE ota_bus_company (
    bus_company_id                INTEGER PRIMARY KEY,
    bus_company_name              VARCHAR2(30),
    bus_company_cnpj              VARCHAR2(18)
);

In [0]:
import pandas as pd

df_sand_bus_company = spark.table("bus.bronze.ota_bus_ticket_sales")

unique_place_origin_departure = (
    df_sand_bus_company
    .select("place_origin_departure")
    .dropna(subset=["place_origin_departure"])
    .distinct()
    .toPandas()["place_origin_departure"]
)

unique_place_destination_departure = (
    df_sand_bus_company
    .select("place_destination_departure")
    .dropna(subset=["place_destination_departure"])
    .distinct()
    .toPandas()["place_destination_departure"]
)

unique_place_origin_return = (
    df_sand_bus_company
    .select("place_origin_return")
    .dropna(subset=["place_origin_return"])
    .distinct()
    .toPandas()["place_origin_return"]
)

unique_place_destination_return = (
    df_sand_bus_company
    .select("place_destination_return")
    .dropna(subset=["place_destination_return"])
    .distinct()
    .toPandas()["place_destination_return"]
)

unique_companies = pd.Series(
    list(unique_place_origin_departure) + list(unique_place_destination_departure) + list(unique_place_origin_return)
    + list(unique_place_destination_return)
).drop_duplicates().reset_index(drop=True)

ota_bus_places_data = pd.DataFrame({
    "bus_places_id": range(1, len(unique_companies) + 1),
    "bus_places_name": unique_companies
})

df_gold_bus_places = spark.createDataFrame(ota_bus_places_data)

df_gold_bus_places.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bus.gold.ota_bus_places_data")

In [0]:
%sql
CREATE TABLE ota_bus_places_data (
    bus_places_id                INTEGER PRIMARY KEY,
    bus_places_name              VARCHAR2(55)
);

**Documentação da Tabela:**

Este comando SQL cria a tabela ota_bus_ticket_sales no banco de dados Oracle. A tabela armazena o histórico de compras de passagens de ônibus dos clientes, sendo uma tabela de fatos em um modelo de Data Warehouse. Cada registro nesta tabela representa uma transação de compra de passagem.

In [0]:
%sql
CREATE TABLE ota_bus_ticket_sales (
    nk_ota_localizer_id             VARCHAR2(20) PRIMARY KEY,
    fk_contact                      VARCHAR2(20),
    date_purchase                   DATE,
    time_purchase                   VARCHAR2(10),
    cod_place_origin_departure      INTEGER,
    place_origin_departure          VARCHAR2(100),
    cod_place_destination_departure INTEGER,
    place_destination_departure     VARCHAR2(100),
    cod_place_origin_return         INTEGER,
    place_origin_return             VARCHAR2(100),
    cod_place_destination_return    INTEGER,
    place_destination_return        VARCHAR2(100),
    cod_departure_bus_company       INTEGER,
    fk_departure_ota_bus_company    VARCHAR2(100),
    cod_return_bus_company          INTEGER,
    fk_return_ota_bus_company       VARCHAR2(100),
    gmv_success                     NUMBER(10,2),
    total_tickets_quantity_success  NUMBER(5),

    CONSTRAINT fk_contact FOREIGN KEY (fk_contact) 
        REFERENCES ota_bus_contact_data(fk_contact),

    CONSTRAINT fk_origin_departure_place FOREIGN KEY (cod_place_origin_departure) 
        REFERENCES ota_bus_places_data(bus_places_id),
    CONSTRAINT fk_destination_departure_place FOREIGN KEY (cod_place_destination_departure) 
        REFERENCES ota_bus_places_data(bus_places_id),
    CONSTRAINT fk_origin_return_place FOREIGN KEY (cod_place_origin_return) 
        REFERENCES ota_bus_places_data(bus_places_id),
    CONSTRAINT fk_destination_return_place FOREIGN KEY (cod_place_destination_return) 
        REFERENCES ota_bus_places_data(bus_places_id),

    CONSTRAINT fk_departure_company FOREIGN KEY (cod_departure_bus_company) 
        REFERENCES ota_bus_company(bus_company_id),
    CONSTRAINT fk_return_company FOREIGN KEY (cod_return_bus_company) 
        REFERENCES ota_bus_company(bus_company_id)
);

## Script DML – Inserção de Dados

Estrutura contendo os comandos para popular as tabelas com dados válidos (mínimo de 15 linhas por tabela).

In [0]:
%sql
INSERT ALL

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (1, 1001, 'João Silva', TO_DATE('1985-02-15', 'YYYY-MM-DD'), 'Masculino', 'Casado', 'joao.silva@email.com', '11987654321', 'Rua A, 123', 'Centro', 'São Paulo', 'SP', '01000-000', 'Brasil', 'Premium', 5000.00, '35-44', 'Engenheiro', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (2, 1002, 'Maria Oliveira', TO_DATE('1990-08-22', 'YYYY-MM-DD'), 'Feminino', 'Solteiro', 'maria.oliveira@email.com', '11976543210', 'Rua B, 456', 'Jardim', 'Campinas', 'SP', '13000-000', 'Brasil', 'Básico', 2500.00, '25-34', 'Analista', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (3, 1003, 'Carlos Souza', TO_DATE('1978-11-10', 'YYYY-MM-DD'), 'Masculino', 'Divorciado', 'carlos.souza@email.com', '11975321098', 'Rua C, 789', 'Vila Nova', 'Rio de Janeiro', 'RJ', '20000-000', 'Brasil', 'Regular', 3500.00, '45-54', 'Médico', 'Inativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (4, 1004, 'Ana Costa', TO_DATE('1995-05-05', 'YYYY-MM-DD'), 'Feminino', 'Solteiro', 'ana.costa@email.com', '11976543212', 'Rua D, 101', 'São João', 'Belo Horizonte', 'MG', '30000-000', 'Brasil', 'Novato', 1500.00, '18-24', 'Estudante', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (5, 1005, 'Lucas Pereira', TO_DATE('1982-06-18', 'YYYY-MM-DD'), 'Masculino', 'Casado', 'lucas.pereira@email.com', '11974321098', 'Rua E, 321', 'Lagoa', 'Porto Alegre', 'RS', '90000-000', 'Brasil', 'Premium', 7000.00, '35-44', 'Professor', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (6, 1006, 'Juliana Martins', TO_DATE('1989-01-13', 'YYYY-MM-DD'), 'Feminino', 'Divorciado', 'juliana.martins@email.com', '11973210987', 'Rua F, 654', 'Vila Rica', 'Curitiba', 'PR', '80000-000', 'Brasil', 'Regular', 4000.00, '25-34', 'Advogada', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (7, 1007, 'Eduardo Lima', TO_DATE('1980-04-20', 'YYYY-MM-DD'), 'Masculino', 'Casado', 'eduardo.lima@email.com', '11972109876', 'Rua G, 987', 'Boa Vista', 'Florianópolis', 'SC', '88000-000', 'Brasil', 'Premium', 6000.00, '35-44', 'Arquiteto', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (8, 1008, 'Fernanda Almeida', TO_DATE('1993-07-30', 'YYYY-MM-DD'), 'Feminino', 'Solteiro', 'fernanda.almeida@email.com', '11971098765', 'Rua H, 111', 'Boa Esperança', 'Fortaleza', 'CE', '60000-000', 'Brasil', 'Básico', 2200.00, '25-34', 'Recepcionista', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (9, 1009, 'Roberto Santos', TO_DATE('1975-09-03', 'YYYY-MM-DD'), 'Masculino', 'Casado', 'roberto.santos@email.com', '11969876543', 'Rua I, 222', 'Centro', 'Salvador', 'BA', '40000-000', 'Brasil', 'Regular', 4500.00, '45-54', 'Gerente', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (10, 1010, 'Patrícia Silva', TO_DATE('1997-12-12', 'YYYY-MM-DD'), 'Feminino', 'Solteiro', 'patricia.silva@email.com', '11968765432', 'Rua J, 333', 'Vila São João', 'São Paulo', 'SP', '11000-000', 'Brasil', 'Novato', 1700.00, '18-24', 'Design', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (11, 1011, 'Renato Rocha', TO_DATE('1991-10-19', 'YYYY-MM-DD'), 'Masculino', 'Solteiro', 'renato.rocha@email.com', '11967654321', 'Rua K, 444', 'Centro', 'São Luís', 'MA', '65000-000', 'Brasil', 'Básico', 2600.00, '25-34', 'Técnico', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (12, 1012, 'Sônia Costa', TO_DATE('1983-03-17', 'YYYY-MM-DD'), 'Feminino', 'Casado', 'sonia.costa@email.com', '11976543123', 'Rua L, 555', 'Jardim das Flores', 'Natal', 'RN', '59000-000', 'Brasil', 'Premium', 7500.00, '35-44', 'Psicóloga', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (13, 1013, 'Paulo Ferreira', TO_DATE('1972-05-25', 'YYYY-MM-DD'), 'Masculino', 'Divorciado', 'paulo.ferreira@email.com', '11965432123', 'Rua M, 666', 'Vila dos Alpes', 'São Bernardo do Campo', 'SP', '11010-000', 'Brasil', 'Regular', 3800.00, '45-54', 'Motorista', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (14, 1014, 'Cláudia Mendes', TO_DATE('1987-11-02', 'YYYY-MM-DD'), 'Feminino', 'Casado', 'claudia.mendes@email.com', '11964321098', 'Rua N, 777', 'Vila Mariana', 'São Paulo', 'SP', '08000-000', 'Brasil', 'Básico', 2800.00, '25-34', 'Assistente', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')

  INTO ota_bus_contact_data (fk_contact, customer_code, customer_name, birth_date, gender, marital_status, email, phone_number, address, neighborhood, city, state, postal_code, country, customer_group, income, age_group, occupation, customer_status, last_update_date, active_record_indicator)
  VALUES (15, 1015, 'Gustavo Almeida', TO_DATE('1992-01-01', 'YYYY-MM-DD'), 'Masculino', 'Solteiro', 'gustavo.almeida@email.com', '11965478901', 'Rua O, 888', 'Jardim Nova Esperança', 'Rio de Janeiro', 'RJ', '21000-000', 'Brasil', 'Novato', 1900.00, '25-34', 'Desenvolvedor', 'Ativo', TO_DATE('2025-08-01', 'YYYY-MM-DD'), 'S')
  
SELECT * FROM dual;


In [0]:
%sql
INSERT ALL
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (101, 'Rod Rotas', '12345678000199')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (203, 'Expresso São José', '23456789000188')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (305, 'Viaje Fácil', '34567890000177')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (487, 'Transporte Alegre', '45678901000166')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (562, 'Rota do Sol', '56789012000155')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (619, 'Expresso Paulista', '67890123000144')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (702, 'TransBrasil', '78901234000133')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (815, 'Viação Central', '89012345000122')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (930, 'Expresso Vitória', '90123456000111')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1045, 'Turismo Rápido', '01234567000199')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1153, 'Viação Nova Era', '12345678000188')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1267, 'Rota Universitária', '23456789000177')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1398, 'Transporte Estrela', '34567890000166')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1420, 'Viação Planalto', '45678901000155')
  INTO ota_bus_company (bus_company_id, bus_company_name, bus_company_cnpj) VALUES (1552, 'Expresso da Serra', '56789012000144')
SELECT * FROM dual;

In [0]:
%sql
INSERT ALL
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (101, 'Aracaju')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (203, 'Blumenau')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (305, 'Uberlândia')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (402, 'Londrina')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (563, 'Ponta Grossa')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (612, 'Ribeirão Preto')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (714, 'Santos')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (829, 'Juiz de Fora')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (930, 'Caxias do Sul')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1042, 'Vitória da Conquista')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1156, 'Marília')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1273, 'Cuiabá')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1384, 'Rio Branco')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1490, 'Foz do Iguaçu')
  INTO ota_bus_places_data (bus_places_id, bus_places_name) VALUES (1502, 'Caruaru')
SELECT * FROM dual;

In [0]:
%sql
INSERT ALL

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('3HOYV76C74V8', '16SHI8GNRK', TO_DATE('2020-11-09', 'YYYY-MM-DD'), '17:49:43', 'Natal', 'São Luís', 'São Luís', 'Natal', 'Rod Rotas', 'Expresso Itamarati', 494.12, 5)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('AOSB22L15PKE', 'MND8T41ZEO', TO_DATE('2023-11-21', 'YYYY-MM-DD'), '17:59:08', 'Fortaleza', 'Teresina', 'Teresina', 'Fortaleza', '1001', 'Rod Rotas', 79.55, 3)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('V5EX22TF5GQT', '79O6OXVZMX', TO_DATE('2024-03-26', 'YYYY-MM-DD'), '20:03:08', 'Rio de Janeiro', 'Brasília', 'Brasília', 'Rio de Janeiro', 'Gontijo', 'Expresso Itamarati', 348.65, 4)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('J0BS42M2F0JJ', 'XRVTKS7LAH', TO_DATE('2021-01-19', 'YYYY-MM-DD'), '06:03:12', 'Teresina', 'Campo Grande', 'Campo Grande', 'Teresina', 'Novo Horizonte', 'Aviação Garcia', 330.4, 4)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('BSS5ZRCD4I5K', '292U1RT7H9', TO_DATE('2020-05-31', 'YYYY-MM-DD'), '18:48:17', 'Porto Alegre', 'Brasília', '0', 'Porto Alegre', 'Gontijo', 'Expresso Guanabara', 193.85, 5)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('TLLMQ4UXXX5Y', '3BS9C56SLH', TO_DATE('2023-01-06', 'YYYY-MM-DD'), '01:35:01', 'Campo Grande', 'Cuiabá', 'Cuiabá', 'Campo Grande', 'Eucat', '1001', 204.4, 3)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('UIEG2RHC5E38', 'B4JJLCADAB', TO_DATE('2022-04-28', 'YYYY-MM-DD'), '01:44:34', 'Campo Grande', 'Belém', 'Belém', 'Campo Grande', 'Novo Horizonte', 'Gontijo', 124.41, 1)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('TYJ7GWBLS5ST', 'AR1I89O8A7', TO_DATE('2024-11-10', 'YYYY-MM-DD'), '03:23:17', 'Porto Alegre', 'Curitiba', '0', '0', 'Expresso Guanabara', '1001', 112.33, 2)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('RZVM1YEATUZ0', '08V9L6N4H5', TO_DATE('2020-02-21', 'YYYY-MM-DD'), '11:57:37', 'Campo Grande', 'Fortaleza', 'Fortaleza', '0', 'Águia Branca', 'Gontijo', 255.78, 2)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('ZMLLA37ARHR3', 'G2G0Q87MIX', TO_DATE('2023-04-02', 'YYYY-MM-DD'), '00:59:30', 'Teresina', 'Brasília', '0', 'São Paulo', 'Expresso Itamarati', '1', 238.24, 4)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('BOJ8WWL3P9QR', 'T06QDVZLWF', TO_DATE('2023-06-19', 'YYYY-MM-DD'), '22:32:12', 'Porto Alegre', 'Teresina', 'Porto Alegre', '0', 'Aviação Garcia', 'Expresso Guanabara', 193.27, 1)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('IVPSVZ9866LP', 'BYTOVLFUDO', TO_DATE('2020-05-19', 'YYYY-MM-DD'), '08:18:05', 'João Pessoa', 'São Paulo', '0', '0', 'Aviação Garcia', 'Expresso Itamarati', 136.35, 5)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('JS5U4948EHWI', 'CCGTJAXTOH', TO_DATE('2023-03-22', 'YYYY-MM-DD'), '18:45:53', 'Manaus', 'Curitiba', 'Curitiba', 'Manaus', 'Aviação Garcia', 'Águia Branca', 365.7, 3)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('TAOLFPEMN4BE', 'WM52U3CM3L', TO_DATE('2023-10-21', 'YYYY-MM-DD'), '01:52:44', 'Campo Grande', 'Campinas', 'Campinas', '0', 'Cometa', 'Águia Branca', 346.83, 1)

  INTO ota_bus_ticket_sales (nk_ota_localizer_id, fk_contact, date_purchase, time_purchase, place_origin_departure, place_destination_departure, place_origin_return, place_destination_return, fk_departure_ota_bus_company, fk_return_ota_bus_company, gmv_success, total_tickets_quantity_success)
  VALUES ('QPRVMUY8XWME', 'XS3DZAN0V3', TO_DATE('2022-06-21', 'YYYY-MM-DD'), '17:18:33', 'Belém', 'Maceió', 'São Paulo', 'Natal', 'Expresso Guanabara', 'Eucat', 140.52, 2)
  
SELECT * FROM dual;